<a href="https://colab.research.google.com/github/maxrgnt/pythdc2-project2/blob/master/Clean2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Panel Data
import pandas as pd
# System folders
import os
from pathlib import Path

In [4]:
def abbreviate(stateName):
    abrvDict = {'Alaska':'AL',
                'Arizona':'AZ',
                'California':'CA',
                'Idaho':'ID',
                'Maine':'ME',
                'Michigan':'MI',
                'Minnesota':'MN',
                'Montana':'MT',
                'New Mexico':'NM',
                'New York':'NY',
                'North Dakota':'ND',
                'Texas':'TX',
                'Vermont':'VT',
                'Washington':'WA'}
    abrv = ''
    if stateName in abrvDict:
        abrv = abrvDict[stateName]
    return abrv

def safeDrop(df, cols):
    print(df.shape)
    for col in df.columns:
        if col in cols:
            print(f'removing: {col}')
            df.drop([col], axis=1, inplace=True)
    print(df.shape)
    return df

## Border Data

In [37]:
dataPath = Path.joinpath(Path.cwd(),'data','borderCrossing.csv')
df = pd.read_csv(dataPath)
df.sample(3)

,Abrv,State,Longitude,Latitude,Border,Year,Measure,Value
7829,MT,Montana,-111.96,49.00,US-Canada Border,2016,Pedestrians,184
25587,ND,North Dakota,-101.30,49.00,US-Canada Border,2013,Personal Vehicle Passengers,2097
110514,TX,Texas,-100.51,28.71,US-Mexico Border,1996,Train Passengers,420


In [38]:
df['Measure'].value_counts()

Personal Vehicle Passengers    30196
Bus Passengers                 28820
Pedestrians                    28697
Train Passengers               27623
Name: Measure, dtype: int64

In [39]:
# Only interested in Passenger / Pedestrian crossings
people = df['Measure'].str.contains('Passengers|Pedestrians', case = False)

In [40]:
# Check to see how much data frame shrinks after filtering down
print(f'All measures: {df.shape}')
#df = df[people] # This gave index error when re-running, changed to df.loc
df = df.loc[people]
print(f'Just people: {df.shape}')

All measures: (115336, 8)
Just people: (115336, 8)


In [41]:
# Break out Location into latitude and longitude,
                # substring from 'POINT( ' to ')' and split on ' ' grabbing first then second element set as float,
if 'Location' in df.columns:
    df['Latitude'] = df['Location'].str[len('POINT ('):-1].str.split(' ').str[1].astype(float)
    df['Longitude'] = df['Location'].str[len('POINT ('):-1].str.split(' ').str[0].astype(float)

In [42]:
# drop unneeded rows
# index of all rows where df['Year'] < 1996
print(df.shape)
dropIndex = df.loc[df['Year']>2018].index
df.drop(dropIndex, inplace=True)
print(df.shape)

(115336, 8)
(114588, 8)


In [43]:
# Drop unnecessary columns
df = safeDrop(df, ['Port Code','Port Name','Location','Unnamed: 0','index'])

(114588, 8)
(114588, 8)


In [44]:
# Get state abrv
df['Abrv'] = df['State'].apply(abbreviate)

In [47]:
# Handle the Date column
if 'Date' in df.columns:
    df['newDate'] = pd.to_datetime(df['Date'])
    df['Year'] = df['newDate'].dt.year.astype(int)

In [48]:
# reorganize columns
df = df[['Abrv','State','Longitude','Latitude','Border','Year','Measure','Value']]

In [50]:
df.sample(5)

,Abrv,State,Longitude,Latitude,Border,Year,Measure,Value
27707,AZ,Arizona,-111.54,31.48,US-Mexico Border,2012,Bus Passengers,0
47091,NY,New York,-74.74,44.99,US-Canada Border,2009,Train Passengers,0
53532,ME,Maine,-68.36,44.64,US-Canada Border,2008,Personal Vehicle Passengers,0
17346,ME,Maine,-68.60,47.25,US-Canada Border,2014,Pedestrians,82
100376,ND,North Dakota,-101.63,49.00,US-Canada Border,1999,Personal Vehicle Passengers,1733


In [51]:
# Remove non-pedestrian values to shrink file
df.to_csv(Path.joinpath(Path.cwd(),'data','borderCrossing.csv'), index = False)

## GDP Data

In [5]:
dataPath = Path.joinpath(Path.cwd(),'data','pctChangeGDP.csv')
df = pd.read_csv(dataPath)
df.sample(3)

,State,Abrv,Year,Value
108,North Dakota,ND,2003,9.2
174,Minnesota,MN,2008,2.3
270,Maine,ME,2015,2.9


In [6]:
# rename GeoName to State
df.rename(columns={'GeoName':'State'}, inplace=True)

In [7]:
# Get state abrv
df['Abrv'] = df['State'].apply(abbreviate)

In [8]:
# drop unneeded rows\n",
# index of all rows where df['Abrv'] == ''
print(df.shape)
dropIndex = df.loc[df['Abrv']==''].index
df.drop(dropIndex, inplace=True)
print(df.shape)

(322, 4)
(322, 4)


In [9]:
# drop unneeded rows
# index of all rows where df['Year'] < 1996
print(df.shape)
dropIndex = df.loc[df['Year']<1997].index
df.drop(dropIndex, inplace=True)
print(df.shape)

(322, 4)
(308, 4)


In [10]:
# SAFE DROP
df = safeDrop(df, ['GeoFips'])

(308, 4)
(308, 4)


In [11]:
if 'Year' not in df.columns:
  df = pd.melt(df, id_vars=['State','Abrv'], var_name='Year', value_name = 'Value')

In [12]:
df.sample(5)

,State,Abrv,Year,Value
125,Washington,WA,2004,4.6
164,North Dakota,ND,2007,9.8
97,Washington,WA,2002,3.7
142,California,CA,2006,6.9
266,Alaska,AL,2015,-8.9


In [13]:
# Un-pivoting
df.to_csv(Path.joinpath(Path.cwd(),'data','pctChangeGDP.csv'), index = False)

## Unemployment Data

In [29]:
dataPath = Path.joinpath(Path.cwd(),'data','unemployment.csv')
df = pd.read_csv(dataPath)
df.sample(3)

,State,Year,Abrv,UnemploymentRate,LaborRate
455,Montana,2008,MT,0.053411,0.669954
281,Arizona,1996,AZ,0.058962,0.659505
403,Texas,2004,TX,0.063247,0.673439


In [30]:
# rename GeoName to State
df.rename(columns={'Stata':'State'}, inplace=True)

In [31]:
# get abbreviations
df['Abrv'] = df['State'].apply(abbreviate)
# drop unneeded rows
# index of all rows where df['Abrv'] == ''
print(df.shape)
dropIndex = df.loc[df['Abrv']==''].index
df.drop(dropIndex, inplace=True)
print(df.shape)

(602, 5)
(602, 5)


In [32]:
# drop unneeded rows
# index of all rows where df['Year'] < 1996
print(df.shape)
dropIndex = df.loc[df['Year']<1996].index
df.drop(dropIndex, inplace=True)
print(df.shape)

(602, 5)
(322, 5)


In [33]:
for col in ['Unemployed','Employed','LaborForce','Population']:
    if col in list(df.columns):
        df['UnemploymentRate'] = df['Unemployed'].div(df['Employed'])
        df['LaborRate'] = df['LaborForce'].div(df['Population'])

In [34]:
# drop unneeded columns
df = safeDrop(df, ['FIPS','PercentOfPopulation','PercentOfLaborEmp','PercentOfLaborUnemp','Population','LaborForce','Employed','Unemployed'])

(322, 5)
(322, 5)


In [35]:
df.sample(3)

,State,Year,Abrv,UnemploymentRate,LaborRate
385,Montana,2003,MT,0.049465,0.665608
515,Texas,2012,TX,0.072071,0.655321
361,Texas,2001,TX,0.052111,0.679757


In [36]:
# Un-pivoting
df.to_csv(Path.joinpath(Path.cwd(),'data','unemployment.csv'), index = False)